In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')

/home/sanfermen/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_csv("../datos/bikes_registered_estan.csv", index_col = 0)
df.head()

,instant,season,yr,mnth,holiday,workingday,weathersit,temp,atemp,hum,windspeed,registered,weekday
0,1,0,0,1,1,0,1,14.110847,-0.456399,0.855582,-0.208909,654,0
1,2,0,0,2,0,1,1,14.902598,-0.492863,0.332722,0.687550,670,2
2,3,0,0,3,0,1,2,8.050924,-1.098990,-0.900337,0.685210,1229,2
3,4,1,0,4,0,0,2,8.200000,-1.015201,-0.170632,-0.210435,1454,0
4,5,1,0,5,0,1,2,9.305237,-0.951952,-0.901843,0.060294,1518,0


In [12]:
class Regres_lineal:
    def __init__(self, dataf, col_pred):
        
        self.dataf = dataf
        self.col_pred = col_pred

    def tree_param(self):
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        arbol = DecisionTreeRegressor(random_state =0)
    
        arbol.fit(x_train, y_train)

        max_feat = np.sqrt(len(x_train.columns))
        max_dep = arbol.tree_.max_depth
        return(max_feat, max_dep)

    def metricas(y_te, y_tr, y_te_pred, y_tr_pred, tipo_modelo):
            resultados = {'MAE': [mean_absolute_error(y_te, y_te_pred), mean_absolute_error(y_tr, y_tr_pred)],
                        'MSE': [mean_squared_error(y_te, y_te_pred), mean_squared_error(y_tr, y_tr_pred)],
                        'RMSE': [np.sqrt(mean_squared_error(y_te, y_te_pred)), np.sqrt(mean_squared_error(y_tr, y_tr_pred))],
                        'R2':  [r2_score(y_te, y_te_pred), r2_score(y_tr, y_tr_pred)],
                        "set": ["test", "train"]}
            df = pd.DataFrame(resultados)
            df["modelo"] = tipo_modelo
            return df

    def grid_s_decision(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= DecisionTreeRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)

        def grid_s_forest(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= DecisionTreeRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)

        return mejor_modelo, dt_results

In [5]:
rg = Regres_lineal(df, "registered")

In [6]:
features, depth = rg.tree_param()

In [7]:
features

3.4641016151377544

In [8]:
depth

19

In [10]:
param1 = {"max_depth": [6, 8], 
        "max_features": [3,4],
        "min_samples_split": [50, 100],
        "min_samples_leaf": [50,100]} 

In [11]:
best_model1, result1 = rg.grid_s(param1, "Prueba I", "RandomForestRegressor")

TypeError: grid_s() takes 3 positional arguments but 4 were given

In [36]:
best_model1

DecisionTreeRegressor(max_depth=6, max_features=4, min_samples_leaf=50,
                      min_samples_split=100)

In [37]:
result1

,MAE,MSE,RMSE,R2,set,modelo
0,0.110957,0.019068,0.138087,0.456220,test,Prueba I
1,0.104734,0.019553,0.139833,0.526279,train,Prueba I
